## 将原始数据集自然语言的形式转化为逻辑表达式

In [6]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests
from tqdm import tqdm
import re

## 单条prompt测试

In [2]:
# 文件路径
file_path = "prompts/DataFountain/translation.txt"

# 请求的 URL
url = "https://api.moonshot.cn/v1/files"

headers = {
    'Authorization': 'sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw'
}

# 准备文件数据
files = {'file': open(file_path, 'rb')}

# 发起 POST 请求
response = requests.post(url, headers=headers, files=files)

# 打印响应内容
print(response.status_code)
print(response.text)

200
{"id":"ctdtlc3ul9lnv03m42dg","object":"file","bytes":3351,"created_at":1734073008,"filename":"translation.txt","purpose":"file-extract","status":"ok","status_details":""}


In [3]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

file_object = client.files.create(file=Path("prompts/DataFountain/translation.txt"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text

In [4]:
print(file_content)

{"content":"任务描述：你将被给予一个规则的描述。你的任务是将规则解析为一阶逻辑公式，并输出为可解析的 JSON 格式。\n一阶逻辑公式的语法定义如下：\n1） 确定规则中涉及的所有谓词。\n2） 定义每个谓词的含义和参数。\n3） 根据规则的逻辑结构，使用一阶逻辑的连接词（如∧、∨、⊕、¬、→、↔等）和量词（如∀、∃等）构建逻辑表达式。\n------\n下面是一个例子：\n规则：\n安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。\n###\n{\n    \"rule\": \"安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。\",\n    \"predicates\":\n    [\n        \"指示：Instruction(Leader, Regulation) ::: 表示领导发出的指示和相关规定。\",\n        \"参与：Participate(Inspector, Scene, Task) ::: 表示安全监察专员在事故现场参与某项任务。\",\n        \"赶赴：GoTo(Inspector, Scene) ::: 表示安全监察专员赶赴事故现场。\",\n        \"事故：Accident(Scene) ::: 表示发生在某个场景的事故。\"\n    ],\n    \"logicexpression\":\n    [\n        \"∀s ∀i ∀r (Accident(s) ∧ Instruction(Leader, r) → (GoTo(i, s) ∧ Participate(i, s, EmergencyRescue) ∧ Participate(i, s, Investigation)))\"\n    ]\n}\n------\n下面是一个例子：\n规则：\n交通运输应急保障：安全监管总局建立全国主要危险化学品从业单位的交通地理信息系统。在应急响应时，利用现有的交通资源，协调铁道、民航、军队等系统提供交通支持，协调沿途有关地方人民政府提供交通警戒支持，以保证及时调运危险化学品事故灾难应急救援有关人员、装备、物资。事故发生地省级

In [7]:
import re

with open('data/DF/rules1.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [5]:
print(data[0])

{'rule_id': '1', 'rule_text': '危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。'}


In [6]:
answers = []

In [4]:
import time


# def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
#     retries = 0
#     while retries < max_retries:
#         try:
#             completion = client.chat.completions.create(
#                 model=model,
#                 messages=messages,
#                 temperature=0,
#             )
#             return completion
#         except requests.exceptions.HTTPError as e:
#             print(e.response.status_code)
#             if e.response.status_code == 429:
#                 print('429')
#                 error_data = e.response.json()
#                 if error_data.get('error', {}).get('type') == 'rate_limit_reached_error':
#                     print(f"Rate limit reached. Retrying in {error_data.get('error', {}).get('message', '1')} seconds...")
#                     time.sleep(int(error_data.get('error', {}).get('message').split(' ')[-2]) + 1)
#                     retries += 1
#                 else:
#                     raise  # Re-raise the exception if it's not a rate limit error
#             else:
#                 raise  # Re-raise the exception if it's not a 429 error
#     raise Exception("Max retries reached. API call failed.")

def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            return completion
        except Exception as e:
            if '429' in str(e):  # 检查异常消息中是否包含429
                print("Rate limit reached. Retrying in 1 second...")
                time.sleep(1)  # 等待1秒钟
                retries += 1
            else:
                raise  # Re-raise the exception if it's not a 429 error
    raise Exception("Max retries reached. API call failed.")

In [7]:
# 替换内容
replacement = "上尿路结石包括肾结石和输尿管结石。肾结石分为肾集合管结石、肾盏(肾盏憩室)结石、肾盂结石、鹿角形结石。输尿管结石可分为输尿管上段结石、中段结石及下段结石。"

# 使用正则表达式替换 [[RULES]]
content = re.sub(r'\[\[RULES\]\]', replacement, file_content)

messages = [
    {
        "role": "system",
        "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"
    },
    {
        "role": "system",
        "content": content,
    },
    {
        "role": "user",
        "content": "你好，请根据txt内容生成回答。"
    }
]

completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

print(completion.choices[0].message.content)

根据您提供的规则描述，我们可以将其解析为一阶逻辑公式，并输出为可解析的 JSON 格式。以下是解析结果：

```json
{
    "rule": "上尿路结石包括肾结石和输尿管结石。肾结石分为肾集合管结石、肾盏(肾盏憩室)结石、肾盂结石、鹿角形结石。输尿管结石可分为输尿管上段结石、中段结石及下段结石。",
    "predicates": [
        "上尿路结石：UreteralStone(X) ::: 表示X是上尿路结石。",
        "肾结石：KidneyStone(X) ::: 表示X是肾结石。",
        "输尿管结石：UreteralStone(X) ::: 表示X是输尿管结石。",
        "肾集合管结石：PelvicCalicesStone(X) ::: 表示X是肾集合管结石。",
        "肾盏结石：CalicealStone(X) ::: 表示X是肾盏结石。",
        "肾盂结石：RenalPelvisStone(X) ::: 表示X是肾盂结石。",
        "鹿角形结石：StaghornStone(X) ::: 表示X是鹿角形结石。",
        "输尿管上段结石：UpperUreteralStone(X) ::: 表示X是输尿管上段结石。",
        "输尿管中段结石：MiddleUreteralStone(X) ::: 表示X是输尿管中段结石。",
        "输尿管下段结石：LowerUreteralStone(X) ::: 表示X是输尿管下段结石。"
    ],
    "logicexpression": [
        "∀X (UreteralStone(X) → (KidneyStone(X) ∨ UreteralStone(X)))",
        "∀X (KidneyStone(X) → (PelvicCalicesStone(X) ∨ CalicealStone(X) ∨ RenalPelvisStone(X) ∨ StaghornStone(X)))",
        "∀X (UreteralStone(X) → (UpperUreteralStone(X) ∨ MiddleUreteralStone(X) ∨ Lowe

In [13]:
for dicts in tqdm(data[84], desc='rule转化逻辑表达式进度'):
    # 替换内容
    replacement = dicts['rule_text']

    # 使用正则表达式替换 [[RULES]]
    content = re.sub(r'\[\[RULES\]\]', replacement, file_content)

    messages = [
        {
            "role": "system",
            "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。请将内容输出为可解析的json格式。"
        },
        {
            "role": "system",
            "content": content,
        },
        {
            "role": "user",
            "content": "你好，请根据txt内容生成回答。"
        }
    ]

    completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

    print(dicts['rule_id'])
    answers.append(completion.choices[0].message.content)

rule转化逻辑表达式进度:   0%|          | 1/618 [00:23<4:01:50, 23.52s/it]

183


rule转化逻辑表达式进度:   0%|          | 2/618 [00:46<3:59:04, 23.29s/it]

184


rule转化逻辑表达式进度:   0%|          | 3/618 [01:15<4:26:09, 25.97s/it]

...

798


rule转化逻辑表达式进度: 100%|█████████▉| 617/618 [5:20:59<00:22, 22.55s/it]

799


rule转化逻辑表达式进度: 100%|██████████| 618/618 [5:21:16<00:00, 31.19s/it]

800


In [14]:
with open('data/DF/answers_v3.json', 'w', encoding='utf-8') as f:
    json.dump(answers, f, ensure_ascii=False, indent=4)

In [10]:
print(answers[0])

根据提供的规则描述，我们可以将其解析为一阶逻辑公式。以下是规则的逻辑表达式：

```json
{
    "rule": "危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。",
    "predicates": [
        "危险化学品事故：ChemicalAccident(Cause, Injury, PropertyDamage, EnvironmentalPollution) ::: 表示由危险化学品造成的事故，包括人员伤害、财产损失和环境污染。",
        "造成：Cause(Accident, Effect) ::: 表示事故造成了某种效果。",
        "人员伤害：Injury(Person) ::: 表示事故中人员受到伤害。",
        "财产损失：PropertyDamage(Asset) ::: 表示事故中财产受到损失。",
        "环境污染：EnvironmentalPollution(Environment) ::: 表示事故中环境受到污染。",
        "生产：Produce(Chemical) ::: 表示危险化学品的生产过程。",
        "经营：Operate(Chemical) ::: 表示危险化学品的经营过程。",
        "储存：Store(Chemical) ::: 表示危险化学品的储存过程。",
        "运输：Transport(Chemical) ::: 表示危险化学品的运输过程。",
        "使用：Use(Chemical) ::: 表示危险化学品的使用过程。",
        "废弃处置：Dispose(Chemical) ::: 表示危险化学品的废弃处置过程。",
        "矿山开采事故：MiningAccident(GasPoisoning, Explosion, Blasting) ::: 表示矿山开采过程中发生的事故，包括有毒、有害气体中毒、爆炸事故、放炮事故。"
    ],
    "logicexpression": [
        "∀c (Chemi

## cache方法传输测试

In [4]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

In [ ]:
def upload_files(files: List[str], cache_tag: Optional[str] = None) -> List[Dict[str, Any]]:
    """
    upload_files 会将传入的文件（路径）全部通过文件上传接口 '/v1/files' 上传，并获取上传后的
    文件内容生成文件 messages。每个文件会是一个独立的 message，这些 message 的 role 均为
    system，Kimi 大模型会正确识别这些 system messages 中的文件内容。

    如果你设置了 cache_tag 参数，那么 upload_files 还会将你上传的文件内容存入 Context Cache
    上下文缓存中，后续你就可以使用这个 Cache 来对文件内容进行提问。当你指定了 cache_tag 的值时，
    upload_files 会生成一个 role 为 cache 的 message，通过这个 message，你可以引用已被缓存
    的文件内容，这样就不必每次调用 `/v1/chat/completions` 接口时都要把文件内容再传输一遍。

    注意，如果你设置了 cache_tag 的值，你需要把 upload_files 返回的 messages 放置在请求
    `/v1/chat/completions` 接口时 messages 参数列表的第一位（实际上，我们推荐不管是否启用
    cache_tag，都将 upload_files 返回的 messages 放置在 messages 列表的头部）。

    关于 Context Caching 的具体信息，可以访问这里：

    https://platform.moonshot.cn/docs/api/caching

    :param files: 一个包含要上传文件的路径的列表，路径可以是绝对路径也可以是相对路径，请使用字符串
        的形式传递文件路径。
    :param cache_tag: 设置 Context Caching 的 tag 值，你可以将 tag 理解为自定义的 Cache 名称，
        当你设置了 cache_tag 的值，就意味着启用 Context Caching 功能，默认缓存时间是 300 秒，每次
        携带缓存进行 `/v1/chat/completions` 请求都将刷新缓存存活时间（300 秒）。
    :return: 一个包含了文件内容或文件缓存的 messages 列表，请将这些 messages 加入到 Context 中，
        即请求 `/v1/chat/completions` 接口时的 messages 参数中。
    """
    messages = []

    # 对每个文件路径，我们都会上传文件并抽取文件内容，最后生成一个 role 为 system 的 message，并加入
    # 到最终返回的 messages 列表中。
    for file in files:
        file_object = client.files.create(file=Path(file), purpose="file-extract")
        file_content = client.files.content(file_id=file_object.id).text
        messages.append({
            "role": "system",
            "content": file_content,
        })

    if cache_tag:
        # 当启用缓存（即 cache_tag 有值时），我们通过 HTTP 接口创建缓存，缓存的内容则是前文中通过文件上传
        # 和抽取接口生成的 messages 内容，我们为这些缓存设置一个默认的有效期 300 秒（通过 ttl 字段），并
        # 为这个缓存打上标记，标记值为 cache_tag（通过 tags 字段）。
        r = httpx.post(f"{client.base_url}caching",
                       headers={
                           "Authorization": f"Bearer {client.api_key}",
                       },
                       json={
                           "model": "moonshot-v1",
                           "messages": messages,
                           "ttl": 300,
                           "tags": [cache_tag],
                       })

        if r.status_code != 200:
            raise Exception(r.text)

        # 创建缓存成功后，我们不再需要将文件抽取后的内容原封不动地加入 messages 中，取而代之的是，我们可以设置一个
        # role 为 cache 的消息来引用我们已缓存的文件内容，只需要在 content 中指定我们给 Cache 设定的 tag 即可，
        # 这样可以有效减少网络传输的开销，即使是多个文件内容，也只需要添加一条 message，保持 messages 列表的清爽感。
        return [{
            "role": "cache",
            "content": f"tag={cache_tag};reset_ttl=300",
        }]
    else:
        return messages